In [1]:
import rpyc
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
# m = motor.LargeMotor('outA')
# m.run_timed(time_sp=1000, speed_sp=-1050)

In [2]:
def get_motors_and_sensor(motor, sensor):
    cs = sensor.ColorSensor('in4')
    ts1 = sensor.TouchSensor('in1')
    ts2 = sensor.TouchSensor('in2')
    bot = motor.LargeMotor('outA')
    top = motor.LargeMotor('outB')
    return top, bot, cs, ts1, ts2

def record_scan_path():
    top_l = []
    bot_l = []
    top.reset(), bot.reset()
    #winsound.Beep(400, 500)
    #winsound.Beep(400, 500)
    #winsound.Beep(600, 500)
    while(ts1.value() == 0):
        while(ts2.value() == 0):
            pass
        top_p, bot_p = top.position, bot.position
        print(top.position, bot.position)
        top_l.append(top_p)
        bot_l.append(bot_p)
    return _clip_instructions(top_l, bot_l)

def _clip_instructions(top_l, bot_l):
    prev_t, prev_b = top_l[0], bot_l[0]
    remove_front = 0
    for t, b in zip(top_l, bot_l):
        if t == prev_t and b == prev_b:
            remove_front += 1        
        else:
            break
            
    prev_t, prev_b = top_l[-1], bot_l[-1]
    remove_back = 0
    for t, b in zip(reversed(top_l), reversed(bot_l)):
        if t == prev_t and b == prev_b:
            remove_back += 1        
        else:
            break
    
    first = remove_front - 1
    if first < 0:
        first = 0
    last = -remove_back + 1
    if last >= 0:
        last = None
    
    return top_l[first:last], bot_l[first:last]

In [3]:
# bot.run_timed(time_sp=250, speed_sp=100)
# top.run_timed(time_sp=250, speed_sp=100)

In [4]:
conn = rpyc.classic.connect('ev3dev.local')
top, bot, cs, ts1, ts2 = get_motors_and_sensor(conn.modules['ev3dev2.motor'], conn.modules['ev3dev2.sensor.lego'])

In [ ]:
top_l, bot_l = record_scan_path()

In [ ]:
np.save('top_l.npy', top_l)
np.save('bot_l.npy', bot_l)

In [ ]:
top_l, bot_l = np.load('top_l.npy'), np.load('bot_l.npy')

In [ ]:
top.run_to_abs_pos(position_sp=0)
bot.run_to_abs_pos(position_sp=0)

In [ ]:
top.stop(), bot.stop(), top.speed_sp

In [ ]:
WHITE = (188, 217, 240)

In [ ]:
def reset_position():
    top.run_to_abs_pos(position_sp = 0)
    bot.run_to_abs_pos(position_sp = 0)
    if top.wait_until_not_moving(timeout=2000) and bot.wait_until_not_moving(timeout=2000):
        pass

def clip_to_white(rgb):
    def clip_one(value, max_):
        return np.max(value, max_)
    return [(np.max([r, WHITE[0]]), np.max([g, WHITE[1]]), np.max([b, WHITE[2]])) for r,g,b in rgb ]
    
def get_reading(top_l, bot_l, lim_n, timeout=2000):
    reading = []
    for (a, b) in tqdm(zip(top_l[:lim_n],bot_l[:lim_n])):
        bot.run_to_abs_pos(position_sp = b)
        top.run_to_abs_pos(position_sp = a)
        reading.append(cs.raw)
        reading.append(cs.raw)
        reading.append(cs.raw)
        reading.append(cs.raw)
        reading.append(cs.raw)
        reading.append(cs.raw)
        if top.wait_until_not_moving(timeout=timeout) and bot.wait_until_not_moving(timeout=timeout):
            pass
        reading.append(cs.raw)
        reading.append(cs.raw)
    return clip_to_white(reading)

def perform_readings(top_l, bot_l, n_runs=1, lim_n=None, speed=100):
    top.speed_sp, bot.speed_sp = [speed] * 2
    readings = []
    for _ in range(n_runs):
        reset_position()
        reading = get_reading(top_l, bot_l, lim_n)
        readings.append(reading)
    return np.squeeze(np.array(readings))

def get_rgb_mean_from_multiple_readings(readings):
    return get_rgb(np.mean(readings, axis=0))

def get_rgb(rgb):
    return np.split(rgb, indices_or_sections=3, axis=1)

def plot_rgb(r,g,b):
    plt.plot(r, 'red')
    plt.plot(g, 'green')
    plt.plot(b, 'blue')
    plt.show()

In [ ]:
max_readings = 4
readings = []
stop_flag = False
for _ in range(max_readings):
    while(ts1.value() == 0):
        if(ts2.value() == 1):
            stop_flag = True
            break
        pass
    if stop_flag:
        break
    
    reading = perform_readings(top_l[::], bot_l[::], n_runs=1, lim_n=None, speed=50)
    plot_rgb(*get_rgb(reading))
    readings.append(reading)

In [ ]:
np.save('triangle_rotated.npy', readings)

In [ ]:
max_readings = 3
stop_flag = False
for _ in range(max_readings):
    while(ts1.value() == 0):
        if(ts2.value() == 1):
            stop_flag = True
            break
        pass
    if stop_flag:
        break
    
    reading = perform_readings(top_l[::], bot_l[::], n_runs=1, lim_n=None, speed=50)
    plot_rgb(*get_rgb(reading))

-----

See if we can get speed values from motors

In [ ]:
motor = conn.modules['ev3dev2.motor']      # import ev3dev2.ev3 remotely
sensor = conn.modules['ev3dev2.sensor.lego']
leds = conn.modules['ev3dev2.led']

In [ ]:
#bot.run_forever(speed_sp = -30)
for i in range(1000):
  print('\r' +str(bot.speed), end = ' ')

In [ ]:
import time
speed_arr = []
pos_arr = []
bot.ramp_up_sp = 5
top.ramp_up_sp = 5
print(bot.speed,top.speed,bot.position, top.position)
print('GOGOGOGOGGOGOGO')
time.sleep(.1)
for i in range(100):
  speed_arr.append([bot.speed,top.speed])
  pos_arr.append([bot.position,top.position])
  time.sleep(.1)

In [ ]:
bot.speed_sp = 50
top.speed_sp = 50

bot.speed_p = 2000
top.speed_p = 2000

bot.speed_i = 60
bot.speed_i = 60

bot.speed_d = 0
top.speed_d = 0

for i in range(100):
  bot.run_to_abs_pos(position_sp = pos_arr[i][0])#, speed_sp = speed_arr[i][0])
  top.run_to_abs_pos(position_sp = pos_arr[i][1])#, speed_sp = speed_arr[i][1])
  #bot.run_forever(speed_sp = speed_arr[i][0])
  #top.run_forever(speed_sp = speed_arr[i][1])
  #print(bot.speed_p, bot.speed_i, bot.speed_d)
  time.sleep(.1)

In [ ]:
s_col = sensor.ColorSensor('in4')
col_arr = []
for i in range(100):
  time.sleep(.05)
  col_arr.append(s_col.raw)

In [ ]:
plt.plot(col_arr)
plt.show()

In [7]:
top.run_direct(duty_cycle_sp = 50)

In [32]:
top.duty_cycle_sp = -20


In [34]:
top.reset()

In [ ]:
dir(top)

In [ ]:
top.on_to_position?

In [ ]:
top.reset?

In [ ]:
top.reset(), bot.reset()